# Ron Day 2 Homework - Group Coding Challenge
Exploring the LJ Potential and Cutoffs

In [ ]:
import math

In [ ]:
def calculate_LJ(r_ij): 
    """
    The LJ interaction energy between two particles.
    
    Computes the pairwise Lennard jones interaction energy based on the separation distance in reduced units.
    
    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
        
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.
    
    """
    
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy

In [ ]:
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
fig = plt.figure ()
ax = fig.add_subplot(111)
plt.ylim(-2,50)
fig.show()

In [ ]:
for i in range(1, 51):
    r = i * 0.1
    LJ_energy = calculate_LJ(r)
    ax.plot(r,LJ_energy, 'ob')
    #print(f'{r} \t  {LJ_energy}')

## Is 3sigma a good cutoff value or not?
If we plug a distance of 3sigma into the Lennard Jones equation, the terms reduce to 4E[(1/3)^12 - (1/3)^6] = -0.0055* Epsilon. Since Epsilon in SI units is typically a tiny smaller decimal number, that product works out to be a miniscule value, on an order of magnitude essentially equivalent to 0. Therefore, I agree that a cutoff of 3sigma should be satisfactory.


In [ ]:
def calculate_distance(coord1, coord2):
    """
    Calculate the distance between two 3D coordinates.
    
    Parameters
    ----------
    coord1, coord2 : list
        The atomic coordinates [x, y, z]
        
    Returns
    -------
    distance: float
        The distance between the two atoms
    
    """
    sum_square_diff = 0
    
    for i in range(len(coord1)):
        sum_square_diff += (coord1[i] - coord2[i])**2
        
    distance = math.sqrt(sum_square_diff)
    
    return distance

In [ ]:
def calculate_total_pair_energy(coordinates, cutoff):
    """
    Calculate the total Lennard Jonest energy of a system of particles.
    
    Parameters
    ----------
    coordinates : list
        Nested list containing particle coordinates.
        
    Returns
    -------
    total_energy : float
        The total pairwise Lennard jonest energy of the system of particles.
    """
    
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            # Calculate the distance between the particles - exercise.
            dist_ij = calculate_distance(coordinates[i], coordinates[j])

            if dist_ij < cutoff:
                # Calculate the pairwise LJ energy
                LJ_ij = calculate_LJ(dist_ij)

                # Add to total energy.
                total_energy += LJ_ij
    return total_energy

In [ ]:
def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length

In [ ]:
import os

In [ ]:
%cd ../..

In [ ]:
file_path = os.path.join('lj_sample_configurations', 'lj_sample_config_periodic1.txt')

coordinates, box_length = read_xyz(file_path)

In [ ]:
%time calculate_total_pair_energy(coordinates, 1)

In [ ]:
%time calculate_total_pair_energy(coordinates, 2)

In [ ]:
%time calculate_total_pair_energy(coordinates, 3)